# find index of closest values in numpy array

In [1]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = np.unravel_index(np.abs(array - value).argmin(), array.shape)
    return array[idx],idx

# shift lon from 0-360 to -180,180

In [3]:
.assign_coords({"lon": (((data.lon + 180) % 360) - 180)})

# datetime format from datestring

In [ ]:
datetime.strptime('{}'.format(data.time[i].values), '%Y/%m/%d %H:%M:%S')

# update font size for whole plot

In [ ]:
plt.rcParams.update({'font.size':14})

# get list of all values in array of duplicates

In [ ]:
list(set(data))

# value in two different arrays? -> boolean

In [ ]:
np.in1d(data1,data2)

# cartopy projection

In [ ]:
lat_min = 45
lat_max = 65
lon_min = -60
lon_max = -40
resolution = '10m'
central_lon, central_lat = (lon_max+lon_min)/2, (lat_max-lat_min)/2
extent = [lon_min, lon_max, lat_min, lat_max]
fig,ax = plt.subplots(1,1,figsize=(10,10),subplot_kw=dict(projection=ccrs.Orthographic(central_lon, central_lat)))
ax.coastlines()
ax.set_extent(extent)
gl = ax.gridlines(draw_labels=True,color='grey')
gl.xlocator= mticker.FixedLocator(np.arange(-60,-35,5))

# calculate anomalies in xarray

In [ ]:
data_ano = data.groupby('time.dayofyear') - data_mean.rename({'doy':'dayofyear'})

# custom legend

In [ ]:
from matplotlib.lines import Line2D
colors = ['tab:red', 'tab:blue']
lines = [Line2D([0], [0], color=c, linewidth=3) for c in colors]
labels = [ 'temperature', 'salinity']

# add colorbar axis

In [ ]:
# assign to plot
cax = fig.add_axes([0.25,0.1,0.5,0.033]) 
cbar = fig.colorbar(plot,orientation='horizontal',cax=cax)
cbar.set_label(r'$\sigma_T$ in $^\circ$C',fontsize=14)
cbar.set_ticks(np.arange(vmin,vmax+1,1))

# completely custom
m = plt.cm.ScalarMappable(cmap=cmocean.cm.balance)
m.set_clim(-0.25,0.25)
cax = fig.add_axes([0.25,-0.04,0.5,0.033]) 
cbar = fig.colorbar(m,orientation='horizontal',cax=cax,boundaries=np.arange(-0.25,0.3,0.05))
cbar.set_label(r'velocity [m/s]')
cbar.set_ticks(np.arange(-0.25,0.3,0.05))

# distance between two lat/lon points along section

In [ ]:
distance = [0]
R = 6371e3
lat = ole_orca.lat*np.pi/180
lon = ole_orca.lon*np.pi/180
for i in np.arange(1,78):
    dlat = lat[i] - lat[i-1]
    dlon = lon[i] - lon[i-1]
    a = np.sin(dlat/2)*np.sin(dlat/2) + np.cos(lat[i])*np.cos(lat[i-1]) * np.sin(dlon/2)*np.sin(dlon/2)
    c = 2 * np.arctan2(np.sqrt(a),np.sqrt(1-a))
    distance.append((R*c).values)
model_distance = np.cumsum(distance)/1000

# interpolate on other grid

In [ ]:
bathy_viking = np.full_like(viking[0,:,:], np.nan)
bathy_viking[:,:] = griddata((bathy.nav_lon.values.ravel(), bathy.nav_lat.values.ravel()),
                            bathy[:,:].values.ravel(),
                            (viking.nav_lon.values, viking.nav_lat.values),
                            fill_value=np.nan,
                            method='linear')

# detrend xarray non-uniform data

In [2]:
def demean_xarray(da,dim):
    return da - da.mean(dim=dim),da.mean(dim=dim)

def detrend_xarray(da):
    '''
    apply detrend along time axis of 2D Xarray
    see polynomial_detrend for details about detrending
    '''
    dt = xr.apply_ufunc(
                polynomial_detrend,
                da,
                input_core_dims=[['time']],
                output_core_dims=[['time','dim0']],
                vectorize=True,
                output_dtypes=[da.dtype],
                dask="parallelized",
            )
    if 'x' in da.dims:
        dt = dt.transpose('time','y','x','dim0')
    elif 'lon' in da.dims:
        dt = dt.transpose('time','lat','lon','dim0')
    
    return dt.isel(dim0=0),dt.isel(dim0=1)

def polynomial_detrend(da,order=1):
    '''
    detrend of non uniform data on a uniform grid with nan values
    
    change order to increase order of fittet polynomial, default is 1, so linear
    
    returns detrended data and trend values
    '''
    ds = da.copy()
    mask = ~np.isnan(ds)
    if mask.sum() == 0:
        return np.stack((ds,ds),axis=-1)
    else:
        ds_masked = ds[mask]
        time = np.arange(0,len(ds))
        time_masked = time[mask]
        coeff = np.polyfit(time_masked, ds_masked, order)
        trend_nonan = np.polyval(coeff, time_masked)
        detrended = ds_masked - trend_nonan
        ds[mask] = detrended
        trend = np.copy(ds)
        trend[mask] = trend_nonan
        
        return np.stack((ds.data,trend),axis=-1)

# non-uniform discrete fourier transform

In [4]:
def nufft(data,xarray_apply=True):
    '''
    does a non-uniform fast fourier transform on data with a uniform grid but nan values in it
    
    returns freq: frequency in cycles per timestep (cph for hourly data)
            f_k: amplitude for each wavenumber k as a complex number
            ps: power spectrum
            psd: power spectral density
            
    xarray_apply is used for the spectral_analysis function for apply_ufunc along time dimension for each point of spatial array
        if you just want to do it for one array time series, set it to False
    
    function is taken from https://github.com/jakevdp/nfft
    '''
    mask = ~np.isnan(data)
    
    N_freq = len(data)
    k = -N_freq//2 + np.arange(N_freq)
    
    data_masked = (data)[mask]
    t = np.linspace(0, 1,N_freq)[mask]
    
    f_k = nfft.nfft_adjoint(t,data_masked,N_freq)
    ps = np.abs(f_k)**2/N_freq**2
    psd = ps * N_freq
    freq = k/N_freq
    if xarray_apply == True:
        t_return = np.linspace(0, 1,N_freq)
        t_return[~mask] = np.nan
        return np.stack((freq,f_k,ps,psd,t_return),axis=-1)
    elif xarray_apply==False:
        return freq,f_k,ps,psd,t

def spectral_analysis(da):
    '''
    apply the nufft on each spatial point of a 2D dataset along the timeseries
    returns xarray DataSet with frequency spectrum, powerspectrum, power spectral density for each point
    t is masked time array without nans, necessary for reconstruction of time series from frequency spectrum
    '''
    dt = xr.apply_ufunc(
                nufft,
                da,
                input_core_dims=[['time']],
                output_core_dims=[['freq','dim0']],
                vectorize=True,
                output_dtypes=['complex'],
                dask="parallelized",
                dask_gufunc_kwargs={'output_sizes':{'freq':744,'dim0':5}}
            )
    dt = xr.Dataset(coords={
        'lat':(['y','x'],dt['lat'].data),
        'lon':(['y','x'],dt['lon'].data),
        'freq':np.real(dt[0,0,:,0].data)
    },data_vars={
        'f_k':(['y','x','freq'],dt.isel(dim0=1).data),
        'ps':(['y','x','freq'],np.real(dt.isel(dim0=2).data)),
        'psd':(['y','x','freq'],np.real(dt.isel(dim0=3).data)),
        't':(['y','x','freq'],np.real(dt.isel(dim0=4).data))
        

    })
    dt = dt.transpose('freq','y','x')
    dt = dt.where(dt!=0)
    return dt

# spectral_u = spectral_analysis(velocity.u_detrended)

# xmovie example custom plotfunc

In [ ]:
fig = plt.figure(figsize=(10,10))

def custom_plotfunc(ds,fig,tt,framedim='time',**kwargs):
    ax = fig.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
    ds.isel({framedim:tt}).plot.quiver('lon','lat','u','v',ax=ax)
    ax.add_feature(cfeature.LAND, facecolor='grey',edgecolor='black')
    ax.gridlines(draw_labels=True)
    return None,None
    
# mov = Movie(velocity,custom_plotfunc,input_check=False)
# mov.save('movie_2015.mp4',overwrite_existing=True,framerate=8)

# spatial correlation xarray apply_ufunc

In [ ]:
import xarray as xr
import numpy as np
from scipy import stats
import dask

#stats.personr only works on dataarrays, so select your variable at one point
data = xr.open_dataset('subset.nc').sst 

#maybe mfdataset helps for huge datasets and then parallization; 39 is the size of time, you want all tim points to be in one chunk
# data = xr.open_mfdataset('subset.nc',chunks={'time':39,'lat':100,'lon':100}).sst 

#stats.pearsonr doesnt allow nan inputs, so put nan to 0; it then returns nan again for correlation along constant arrays
data = data.fillna(0)

def correlation(data1,data2):
    # data1 is the spatial field you want to correlate to
    # data2 is your single time series
    # calculates the correlation coefficient and p_value
    # returns the result as a numpy array, because the initial output of the function is of a weird PearsonRResult class, which doesnt work in apply_ufunc
    result = stats.pearsonr(data1,data2)
    return np.stack((result[0],result[1]), axis=-1)

# apply_ufunc takes the function you want to apply and then the necessary input arguments to that function
# so data is your spatial field and then your single time series (I just selected one pointfrom my field)
# the input_core_dimensions basically mean along which dimension your function is applied on
# the output dimension is necesarry because the correlation output is of size 2
# dask='parallelized' makes it faster, but needs some additional arguments for your output

result = xr.apply_ufunc(correlation,data,data.isel(lat=50,lon=50),input_core_dims=[['time'],['time']],output_core_dims=[['statistic']],vectorize=True,
                        dask='parallelized',output_dtypes=[np.dtype(float)],dask_gufunc_kwargs={'output_sizes':{'statistic':2}})

# make xarray dataset of the output, because the output has r and p along one extra dimension, so assign them to single variables
statistics = xr.Dataset(coords={'lat':result.lat,'lon':result.lon}, data_vars = {
    'corrcoef':result[:,:,0],
    'p_value':result[:,:,1]
})

# necessary if you use mfdatasets, so you finally compute the correlation for each chunk
# statistics = statistics.compute()

statistics

# find peaks in spectrum

In [12]:
def peak_finder(freq_data,ps_data,freq_min,freq_max,peak_height):
    '''
    find amplitude and frequency of maximum peak in frequency range of spectrum
    freq_data, ps_data: frequency values and powerspectrum
    freq_min, freq_max: bounds of frequency range
    peak_height: threshold for peak to detect
    
    if you want to detect on frequency amplitudes or power spectral density, adjust peak_height accordingly, should work
    
    returns maximum peak and according frequency
    '''
    
    freq_mask = np.array([freq_data > freq_min]) & np.array([freq_data < freq_max])
    freq_mask = np.squeeze(freq_mask)
    peaks,peak_heights = scipy.signal.find_peaks(ps_data[freq_mask],height=peak_height)
    freq_peaks = freq_data[freq_mask][peaks]
    if len(peak_heights['peak_heights']) != 0:
        peak_max = max(peak_heights['peak_heights'])
        freq_max = freq_peaks[peak_heights['peak_heights'] == peak_max]
    else:
        peak_max,freq_max = np.nan,np.nan
    
    return np.array([freq_max,peak_max])

# spectral ellipses

In [233]:
def ellipse_details(fm,fp):
    '''
    calculates major and minor axis as well as azimuth angle of spectral ellipse from positive and negative frequency amplitudes
    see Walden2012 for mathematical insights
    '''
    Rp = np.abs(fp) + np.abs(fm)
    Rm = np.abs(np.abs(fp) - np.abs(fm))

    major = np.abs(Rp + Rm)
    minor = np.abs(Rp - Rm)

    azimuth = cmath.phase(fp*fm)/2
    return major,minor,azimuth

major,minor,azimuth = ellipse_details(fm,fp)
lonlat = [fm.lon.data ,  fm.lat.data]
rot = int(np.abs(fp).data > np.abs(fm).data)
ell = mpl.patches.Ellipse(xy=lonlat, width=major/1e4, height=minor/1e4, angle = azimuth*180/np.pi,fc='none',ec=['b','r'][rot],transform=ccrs.PlateCarree())

ax.add_patch(ell)
ax.set_aspect('equal')
ax.autoscale()

# spectral time filtering

In [4]:
def hann_window(n,T):
    '''
    calculates the frequency spectrum of a hanning window for n samples leading to n frequencies and T window size
    n has to be integer and n % 2 has to be 0
    '''
    assert n%2==0
    w = np.hanning(T)
    w = w / w.sum()
    freq_hann = np.fft.fftfreq(n)

    w_transform = np.abs(np.fft.fft(w, n=n))
    mid_index = int(n/2)
    amp_hann = np.concatenate((w_transform[mid_index:],w_transform[:mid_index]))
    freq_hann = np.concatenate((freq_hann[mid_index:],freq_hann[:mid_index]))
    return freq_hann,amp_hann

def butterworth_window(n,fc,order):
    '''
    calculates the frequency spectrum of a butterworth window for n samples leading to n frequencies and fc cut off frequency with order of butterworth filter 
    n has to be integer and n % 2 has to be 0
    fs is sample frequency in Hz, we have hourly data, so 1/3600
    '''
    
    assert n%2==0
    fs = 1/3600
    nyq = 0.5 * fs
    fc = fc * (1/3600)
    freqs = (-n//2 + np.arange(n))/n/3600

    b,a = scipy.signal.butter(N=order,Wn=fc,btype='lowpass',fs = fs)
    freq_but, amp_but = scipy.signal.freqz(b,a,fs=fs,worN=freqs)
    return freq_but*3600, amp_but

def inverse_nufft(t,f_k):
    '''
    inverse fourier transform to get back time series from fourier components
    non-uniform discrete fourier transform
    
    returns array of reconstructed velocities on the complete time grid, so with nan values
    '''
    reconstructed = nfft.nfft(drop_na(t), f_k)/len(f_k)
    reconstructed_ontime = np.copy(t)
    reconstructed_ontime[~np.isnan(t)] = reconstructed
    return reconstructed_ontime

def inverse_nufft_window(t,f_k,window_amplitudes):
    '''
    inverse fourier transform to get back time series from fourier components
    non-uniform discrete fourier transform but with amplitude taken out of high frequencies using a hanning window, which has to be predefined
    
    returns array of reconstructed velocities on the complete time grid, so with nan values
    '''
    reconstructed = nfft.nfft(drop_na(t), f_k*window_amplitudes)/len(f_k)
    reconstructed_ontime = np.copy(t)
    reconstructed_ontime[~np.isnan(t)] = reconstructed
    return reconstructed_ontime

# unique values in array but keeps order

In [ ]:
def unique(sequence):
    '''
    finds unique values of array while keeping the order of values in array
    (set() orders values in increasing way)
    '''
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

# section indices

In [ ]:
def section_indices(data,point1,point2):
    '''
    get x,y indices of section between latlon point1 and point2
    
    uses haversine distance to look for closest points on latlon grid to given latlon point
    
    first defines straight latlon line between point1 and point2
    
    then looks for closest points on grids to this line
    
    straight line has higher resolution than grid, so we discard all the x,y points that appear twice
    '''
    lat = np.asarray(data.lat.data)
    lon = np.asarray(data.lon.data)
    
    lat1,lon1 = point1
    lat2,lon2 = point2
    

    f = np.frompyfunc(haversine,4,1)
    value, indices1 = find_nearest(f(lon,lat,lon1,lat1).astype(float),0)
    value, indices2 = find_nearest(f(lon,lat,lon2,lat2).astype(float),0)

    x_dist = np.abs(indices1[1] - indices2[1]) 
    y_dist = np.abs(indices1[0] - indices2[0]) 

    dist_max = max(x_dist,y_dist)

    if (lat1-lat2) == 0:
        longitudes = np.linspace(lon1, lon2, dist_max*2)
        latitudes = np.ones(dist_max*2)*lat1
    elif (lon1-lon2) == 0:
        latitudes = np.linspace(lat1, lat2, dist_max*2)
        longitudes = np.ones(dist_max*2)*lon1
    else:
        latitudes = np.linspace(lat1, lat2, dist_max*2)
        longitudes = (lon2 - lon1)/(lat2 - lat1)*(latitudes - lat1) + lon1

    # x_idx = []
    # y_idx = []
    indices = []
    
    for i in range(dist_max*2):
        dummy = find_nearest(f(lon,lat,longitudes[i],latitudes[i]).astype(float),0)[1]
        # x_idx.append(dummy[1])
        # y_idx.append(dummy[0])
        indices.append((dummy[1],dummy[0]))
    indices = unique(indices)
    x_idx = [x[0] for x in indices]
    y_idx = [x[1] for x in indices]
    
    return x_idx,y_idx,latitudes,longitudes

# rotate velocities on section

In [240]:
def calculate_initial_compass_bearing(pointA, pointB):
    """
    Calculates the bearing between two points.
    The formulae used is the following:
        θ = atan2(sin(Δlong).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
    :Parameters:
      - `pointA: The tuple representing the latitude/longitude for the
        first point. Latitude and longitude must be in decimal degrees
      - `pointB: The tuple representing the latitude/longitude for the
        second point. Latitude and longitude must be in decimal degrees
    :Returns:
      The bearing in degrees
    :Returns Type:
      float
    """
    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

def rotate_velocities(u,v,angle):
    '''
    rotates velocity vector in mathematical positive direction
    '''
    
    u_ = u * np.cos(angle) - v * np.sin(angle)
    v_ = u * np.sin(angle) + v * np.cos(angle)
    
    return u_,v_

def rotate_section(section):
    '''
    rotate velocities on section to cross and along section velocities
    
    !!! This heavily depends on how you defined the section !!!
    
    The initial rotation angle is calculated as the compass bearing between two latlon points. I then substract 90deg and change the sign.
    This leads to a clockwise rotation with the angle being between the cartesian axis and the section. However, the
    initinal angle depends on which one is entered first, leading to a difference of 180deg.
    So check the compass bearing between your point first and think about if this leads to the velocites ending up in the way you want.
    I wanted u to be mostly eastward and v to be mostly northward
    
    '''
    point1 = (section.isel(s=0).lat.data,section.isel(s=0).lon.data)
    point2 = (section.isel(s=-1).lat.data,section.isel(s=-1).lon.data)
    
    angle = np.deg2rad(calculate_initial_compass_bearing(point1,point2))
    angle = angle = -(angle- np.pi/2)
    
    section['cross_vel'],section['along_vel'] = rotate_velocities(section.u_filtered,section.v_filtered,angle)
    return section

# velocity vector time series correlation

In [109]:
def vector_correlation(u1, v1, u2, v2, lag=0, time=None, dt=''):
    """
    calculates the correlation and average veering angle between two velocity 
    vector time series
    following the method from Kundu 1976:
    https://journals.ametsoc.org/view/journals/phoc/6/2/1520-0485_1976_006_0238_evonto_2_0_co_2.xml
    veering angle is counterclockwise angle of the second vector with respect 
    to the first

    lagged correlation shifts first time series in positive time direction

    input:
        velocity component time series as numpy array
        if you want lagged correlation:
            lag: size of lag
            time: time array as np.datetime64[ns] array
            dt: sampling frequency, has to be uniform
    returns:
        correlation and average veering angle in degrees
    example:
        pure eastward and pure northward flow: r=1, angle=90deg
    """

    mask = ~np.isnan(u1)
    u1 = u1[mask][lag:]
    v1 = v1[mask][lag:]

    if lag == 0:
        u2 = u2[mask]
        v2 = v2[mask]
    elif lag != 0:
        # assert time, 'time array needed to do lags'
        u2 = u2[np.in1d(time, time[mask][lag:] - np.timedelta64(lag, dt))]
        v2 = v2[np.in1d(time, time[mask][lag:] - np.timedelta64(lag, dt))]

    w1 = u1 + 1j*v1
    w1 = w1
    w2 = u2 + 1j*v2
    w2 = w2

    rho = np.mean(np.conjugate(w1)*w2)/np.sqrt(np.mean(np.conjugate(w1)*w1))/\
        np.sqrt(np.mean(np.conjugate(w2)*w2))

    return np.array((abs(rho), np.degrees(np.angle(rho))))

# Cross Spectrum for non-uniform time series

In [12]:
def crossSpectrum(x, y, nperseg):
    '''
    calculate the cross spectrum between two time series
    uses a non-uniform FFT for spectral amplitudes
    applies welch's method with a hanning window of size nperseg
    and 50% overlap
    
    input:
        x,y: time series data as numpy array
        nperseg: window size for welch's method
    returns:
        cross-spectral amplitudes
        frequencies
    '''
    cross = np.zeros(nperseg, dtype='complex128')
    for ind in np.arange(0, int(len(x)/nperseg)-0.5, 0.5):
        xp = x[int(ind * nperseg): int((ind + 1)*nperseg)]
        xp = xp - np.mean(xp)
        xp = xp*np.hanning(nperseg)

        yp = y[int(ind * nperseg): int((ind + 1)*nperseg)]
        yp = yp - np.mean(yp)
        yp = yp*np.hanning(nperseg)

        # Do FFT
        freqxi, cfxi, psxi, psdxi, txi = nufft(xp, xarray_apply=False)
        freqyi, cfyi, psyi, psdyi, tyi = nufft(yp, xarray_apply=False)

    # Get cross spectrum
        cross += cfxi.conj()*cfyi
    freq = (-nperseg//2 + np.arange(nperseg))/nperseg
    return np.stack((cross, freq), axis=-1)

# coherence = abs(Pxy)**2/(Pxx*Pyy)

# convert txt of jupyter notebook to .ipynb file

In [2]:
# save txt as .py file

import nbformat.current as convert

conv = convert.read(open('untitled.py', 'r'), 'py')
convert.write(conv, open('Ruehs_Winds.ipynb', 'w'), 'ipynb')